In [1]:
import glob
from PIL import Image
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from datetime import date
from datetime import datetime

D:\programming\Anaconda1\envs\NN\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [2]:
import tensorflow as tf
import numpy as np
import argparse
import math
import os
from matplotlib.image import imread
from scipy.misc.pilutil import imresize, imsave
import matplotlib.pyplot as plt
import cv2

In [3]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Reshape, Flatten, Dropout, Input
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, SGD
from keras.datasets import mnist
from keras import initializers

import numpy as np
import random

%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
def build_generator():
    
        model = Sequential()
        model.add(Dense(128 * 16 * 8, input_dim = latent_dim))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Reshape((8, 8, 256)))
        
        model.add(Conv2DTranspose(512, filter_size_g, strides=(1,1), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(512, filter_size_g, strides=(1,1), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(256, filter_size_g, strides=(1,1), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(128, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(64, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(32, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(16, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(8, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(img_channels, filter_size_g, strides=(1,1), padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        return model

In [ ]:
def build_discriminator():

        model = Sequential()
        
        model.add(Conv2D(64, kernel_size=filter_size_d, strides = (2,2), input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(128, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(256,  kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(512, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(512, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(512, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Flatten())
        model.add(Dense(1))
        model.add(Activation('sigmoid'))

        model.summary()

        return model

In [ ]:
def generator_containing_discriminator(g, d):
    noise = Input(shape=(latent_dim,))
    img = g(noise)
    d.trainable = False
    #d_label.trainable = False
    valid = d(img)
    
    
    return Model(noise, valid)

In [ ]:
def save_imgs(epoch, gen):
        count = 5
        noise = np.random.uniform(-1, 1, size=(count, latent_dim))
        gen_imgs = gen.predict(noise)

        gen_imgs = 127.5 * gen_imgs + 127.5
        
        if gen_imgs.shape[3] == 1:
            gen_imgs = gen_imgs[:,:,:,0]
        
        for i in range(count):
            cv2.imwrite(os.getcwd() + '\\generated\\epoch%d_%d.png' % (epoch, i), gen_imgs[i])

In [ ]:
def get_images(indices, data, batch_size):
    X_train = np.zeros((batch_size, img_rows, img_cols, img_channels))
    for i in range(batch_size):
        if color_mode == 'grayscale':
            temp_img = cv2.imread(data[indices[i]], 0)
            X_train[i,:,:,0] = temp_img
        else:
            temp_img = cv2.imread(data[indices[i]])
            X_train[i] = temp_img
    X_train = (X_train - 127.5)/127.5
    return X_train

In [ ]:
def get_images_classes(batch_size, data):
    X_train = np.zeros((batch_size, img_rows, img_cols, img_channels))
    y_labels = np.zeros(batch_size)
    choice_arr = np.random.randint(0, len(data), batch_size)
    for i in range(batch_size):
        rand_number = np.random.randint(0, len(data[choice_arr[i]]))
        temp_img = cv2.imread(data[choice_arr[i]][rand_number])
        
        
        X_train[i] = temp_img
        y_labels[i] = choice_arr[i]
    

    X_train = (X_train - 127.5)/127.5
    return X_train, y_labels

In [ ]:
def get_images_one_class(batch_size, data, class_target):
    X_train = np.zeros((batch_size, img_rows, img_cols, img_channels))
    y_labels = np.zeros(batch_size) + class_target
    '''choice_arr = np.random.randint(0, len(data[class_target]), batch_size)
    for i in range(batch_size):
        temp_img = cv2.imread(data[class_target][choice_arr[i]])

        X_train[i] = temp_img
        y_labels[i] = choice_arr[i]'''
    
    for i in range(batch_size):
        temp_img = cv2.imread(os.getcwd() + '/new256_images/abstraktnyy-ekspressionizm/303.png')
        
        X_train[i] = temp_img
        y_labels[i] = 0
    X_train = (X_train - 127.5)/127.5
    return X_train, y_labels

In [ ]:
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[1:3]
    image = np.zeros((height*shape[0], width*shape[1], img_channels),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[:, :, :,]
    return image

In [ ]:
def get_data():
    styles_folder = os.listdir(path=os.getcwd() + "\\new256_images\\")
    num_styles = len(styles_folder)
    data = []
    for i in range(num_styles):
        data.append(glob.glob(os.getcwd() + '\\new256_images\\' + styles_folder[i] + '\\*'))
    return data, num_styles

In [ ]:
def train_another(epochs = 100, BATCH_SIZE = 4):

    data, num_styles = get_data()
    
    generator = build_generator()
    discriminator = build_discriminator()
    
    discriminator.compile(loss=losses[0], optimizer=d_optim)
    #d_label.compile(loss=losses[1], optimizer=d_optim)
    generator.compile(loss='binary_crossentropy', optimizer=g_optim)
    
    dcgan = generator_containing_discriminator(generator, discriminator)
    
    
    dcgan.compile(loss=losses[0], optimizer=g_optim)
    
    discriminator.trainable = True
    #d_label.trainable = True
    
    
    '''(X_train, y_train), (X_test, y_test) = mnist.load_data()

    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))'''
    
    
    for epoch in range(epochs):
        for index in range(int(15000/BATCH_SIZE)):
            noise = np.random.normal(0, 1, (BATCH_SIZE, latent_dim))
                
            #real_images, real_labels = get_images_classes(batch_size=BATCH_SIZE, data = data)
            
            label = index % num_styles
            
            real_images, real_labels = get_images_one_class(BATCH_SIZE, data, label)
            
            real_images += np.random.normal(size = img_shape, scale= 0.1)
            
            generated_images = generator.predict(noise)
            
            if index % 2 == 0:
                X = real_images
                real_labels = real_labels - 0.1 + np.random.rand(BATCH_SIZE)*0.2
                y_classif = keras.utils.to_categorical(np.zeros(BATCH_SIZE) + label, num_styles)
                y = 0.8 + np.random.rand(BATCH_SIZE)*0.2
                
                d_loss = []
                d_loss.append(discriminator.train_on_batch(X, y))
                #discriminator.trainable = False
                #d_loss.append(d_label.train_on_batch(X, y_classif))
                print("epoch %d batch %d d_loss : %f" % (epoch, index, d_loss[0]))
                
                X = generated_images
                y = np.random.rand(BATCH_SIZE) * 0.2
                d_loss = discriminator.train_on_batch(X, y)
                
                print("epoch %d batch %d d_loss : %f" % (epoch, index, d_loss))
            else:
                X = generated_images
                y = np.random.rand(BATCH_SIZE) * 0.2
                d_loss = discriminator.train_on_batch(X, y)
                
                X = real_images
                real_labels = real_labels - 0.1 + np.random.rand(BATCH_SIZE)*0.2
                y_classif = keras.utils.to_categorical(np.zeros(BATCH_SIZE) + label, num_styles)
                y = 0.8 + np.random.rand(BATCH_SIZE)*0.2
                d_loss = (discriminator.train_on_batch(X, y))
                
                print("epoch %d batch %d d_loss : %f" % (epoch, index, d_loss))
            
            
            noise = np.random.normal(0, 1, (BATCH_SIZE, latent_dim))
            
            discriminator.trainable = False
            #d_label.trainable = False
            
            y_classif = keras.utils.to_categorical(np.zeros(BATCH_SIZE) + 0.5, num_styles)
            y = np.random.rand(BATCH_SIZE) * 0.3
            
            g_loss = dcgan.train_on_batch(noise, y)
            
            #d_label.trainable = True
            discriminator.trainable = True
            
            print("epoch %d batch %d g_loss : %f" % (epoch, index, g_loss))
            
            if index % 50 == 0:
                        image = combine_images(generated_images)
                        image = image*127.5+127.5
                        cv2.imwrite(
                            os.getcwd() + '\\generated\\epoch%d_%d.png' % (epoch, index), image)
                        image = combine_images(real_images)
                        image = image*127.5+127.5
                        cv2.imwrite(
                            os.getcwd() + '\\generated\\epoch%d_%d_data.png' % (epoch, index), image)
                        
        if epoch % 5 == 0:
            
            
            date_today = date.today()
            
            
            
            month, day = date_today.month, date_today.day
            
            # Генерируем описание модели в формате json
            d_json = discriminator.to_json()
            # Записываем модель в файл
            json_file = open(os.getcwd() + "%d.%d dis_model.json" % (day, month), "w")
            json_file.write(d_json)
            json_file.close()
            
            '''# Генерируем описание модели в формате json
            d_l_json = d_label.to_json()
            # Записываем модель в файл
            json_file = open(os.getcwd() + "%d.%d dis_label_model.json" % (day, month), "w")
            json_file.write(d_l_json)
            json_file.close()'''
            
            # Генерируем описание модели в формате json
            gen_json = generator.to_json()
            # Записываем модель в файл
            json_file = open(os.getcwd() + "%d.%d gen_model.json" % (day, month), "w")
            json_file.write(gen_json)
            json_file.close()
            
            discriminator.save_weights(os.getcwd() + '%d.%d dis_weights.h5' % (day, month))
            #d_label.save_weights(os.getcwd() + '%d.%d dis_label_weights.h5' % (day, month))
            generator.save_weights(os.getcwd() + '%d.%d gen_weights.h5' % (day, month))

In [ ]:
img_rows = 256
img_cols = 256
img_channels = 3
img_shape = (img_rows, img_cols, img_channels)
latent_dim = 100
filter_size_g = (5,5)
filter_size_d = (5,5)
d_strides = (2,2)

color_mode = 'rgb'

losses = ['binary_crossentropy', 'categorical_crossentropy']


#g_optim = SGD(lr = 0.001, momentum=0.9, nesterov=True)
#d_optim = SGD(lr = 0.00025, momentum=0.9, nesterov=True)
g_optim = Adam(0.0002, beta_2 = 0.5)
d_optim = Adam(0.0002, beta_2 = 0.5)

In [ ]:
train_another(1000, 16)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16384)             1654784   
_________________________________________________________________
batch_normalization_1 (Batch (None, 16384)             65536     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16384)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 8, 8, 512)         3277312   
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 8, 512)         2048      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 512)         0         
__________

epoch 0 batch 1 g_loss : 1.303733
epoch 0 batch 2 d_loss : 1.363647
epoch 0 batch 2 d_loss : 1.342477
epoch 0 batch 2 g_loss : 2.127913
epoch 0 batch 3 d_loss : 8.636152
epoch 0 batch 3 g_loss : 2.621328
epoch 0 batch 4 d_loss : 3.433769
epoch 0 batch 4 d_loss : 1.583737
epoch 0 batch 4 g_loss : 2.469853
epoch 0 batch 5 d_loss : 1.435085
epoch 0 batch 5 g_loss : 1.356680
epoch 0 batch 6 d_loss : 1.308934
epoch 0 batch 6 d_loss : 0.449827
epoch 0 batch 6 g_loss : 1.434771
epoch 0 batch 7 d_loss : 1.746174
epoch 0 batch 7 g_loss : 0.820588
epoch 0 batch 8 d_loss : 1.586899
epoch 0 batch 8 d_loss : 1.067900
epoch 0 batch 8 g_loss : 1.423277
epoch 0 batch 9 d_loss : 1.747671
epoch 0 batch 9 g_loss : 1.841674
epoch 0 batch 10 d_loss : 1.472420
epoch 0 batch 10 d_loss : 0.815985
epoch 0 batch 10 g_loss : 0.964755
epoch 0 batch 11 d_loss : 1.787125
epoch 0 batch 11 g_loss : 1.021450
epoch 0 batch 12 d_loss : 1.568441
epoch 0 batch 12 d_loss : 4.103482
epoch 0 batch 12 g_loss : 2.941073
epoch 

epoch 0 batch 95 g_loss : 0.562354
epoch 0 batch 96 d_loss : 0.325979
epoch 0 batch 96 d_loss : 0.483537
epoch 0 batch 96 g_loss : 0.567107
epoch 0 batch 97 d_loss : 0.480738
epoch 0 batch 97 g_loss : 0.446221
epoch 0 batch 98 d_loss : 0.466531
epoch 0 batch 98 d_loss : 0.383300
epoch 0 batch 98 g_loss : 0.439104
epoch 0 batch 99 d_loss : 0.438670
epoch 0 batch 99 g_loss : 0.361872
epoch 0 batch 100 d_loss : 0.321592
epoch 0 batch 100 d_loss : 0.378003
epoch 0 batch 100 g_loss : 0.381661
epoch 0 batch 101 d_loss : 0.584357
epoch 0 batch 101 g_loss : 0.470357
epoch 0 batch 102 d_loss : 0.539615
epoch 0 batch 102 d_loss : 0.442601
epoch 0 batch 102 g_loss : 0.531335
epoch 0 batch 103 d_loss : 0.344257
epoch 0 batch 103 g_loss : 0.582429
epoch 0 batch 104 d_loss : 0.351591
epoch 0 batch 104 d_loss : 0.422082
epoch 0 batch 104 g_loss : 0.375276
epoch 0 batch 105 d_loss : 0.668140
epoch 0 batch 105 g_loss : 0.463188
epoch 0 batch 106 d_loss : 0.422796
epoch 0 batch 106 d_loss : 0.456677
epo

epoch 0 batch 186 g_loss : 0.474193
epoch 0 batch 187 d_loss : 0.325893
epoch 0 batch 187 g_loss : 0.491525
epoch 0 batch 188 d_loss : 0.345306
epoch 0 batch 188 d_loss : 0.371535
epoch 0 batch 188 g_loss : 0.369648
epoch 0 batch 189 d_loss : 0.353672
epoch 0 batch 189 g_loss : 0.470407
epoch 0 batch 190 d_loss : 0.305564
epoch 0 batch 190 d_loss : 0.267376
epoch 0 batch 190 g_loss : 0.607069
epoch 0 batch 191 d_loss : 0.356192
epoch 0 batch 191 g_loss : 0.388038
epoch 0 batch 192 d_loss : 0.331002
epoch 0 batch 192 d_loss : 0.350244
epoch 0 batch 192 g_loss : 0.473267
epoch 0 batch 193 d_loss : 0.399654
epoch 0 batch 193 g_loss : 0.424764
epoch 0 batch 194 d_loss : 0.334980
epoch 0 batch 194 d_loss : 0.431021
epoch 0 batch 194 g_loss : 0.450940
epoch 0 batch 195 d_loss : 0.370476
epoch 0 batch 195 g_loss : 0.524459
epoch 0 batch 196 d_loss : 0.328024
epoch 0 batch 196 d_loss : 0.399700
epoch 0 batch 196 g_loss : 0.570272
epoch 0 batch 197 d_loss : 0.327618
epoch 0 batch 197 g_loss : 0

epoch 0 batch 278 d_loss : 0.382660
epoch 0 batch 278 d_loss : 0.258614
epoch 0 batch 278 g_loss : 0.424284
epoch 0 batch 279 d_loss : 0.369728
epoch 0 batch 279 g_loss : 0.600507
epoch 0 batch 280 d_loss : 0.346475
epoch 0 batch 280 d_loss : 0.313315
epoch 0 batch 280 g_loss : 0.536906
epoch 0 batch 281 d_loss : 0.465716
epoch 0 batch 281 g_loss : 0.432328
epoch 0 batch 282 d_loss : 0.356463
epoch 0 batch 282 d_loss : 0.287438
epoch 0 batch 282 g_loss : 0.517873
epoch 0 batch 283 d_loss : 0.261049
epoch 0 batch 283 g_loss : 0.437416
epoch 0 batch 284 d_loss : 0.350417
epoch 0 batch 284 d_loss : 0.271682
epoch 0 batch 284 g_loss : 0.557264
epoch 0 batch 285 d_loss : 0.323943
epoch 0 batch 285 g_loss : 0.546530
epoch 0 batch 286 d_loss : 0.337671
epoch 0 batch 286 d_loss : 0.352591
epoch 0 batch 286 g_loss : 0.451595
epoch 0 batch 287 d_loss : 0.418411
epoch 0 batch 287 g_loss : 0.514815
epoch 0 batch 288 d_loss : 0.429228
epoch 0 batch 288 d_loss : 0.294953
epoch 0 batch 288 g_loss : 0

In [ ]:
current_moment = datetime.now()

day, month = str(current_moment.day), str(current_moment.month)
hour, minute = str(current_moment.hour), str(current_moment.minute)

with open(os.getcwd() + '/Changes.txt', 'a') as f:
    clock =  (day + '.' + month + '    ' + hour + ':' + minute)
    f.write(clock + '...........g_optim -> 0.0003, d_optim->0.0004' + '\n')